# mTRF Pre-processing

Version 4 · Last update: 09/10/2024

## Overview

The mTRF Pre-processing consists in the following steps, and takes ~1 hr per session:
* [Setup the environment](#setup)
* [Load the data](#load) (~2 min)
* [Bad channel detection](#bad) (~10 min)
* [Maxwell filtering and Signal-space separation](#maxwell) (~15 min) · <span style="color:#ffcc00">Checkpoint #1</span>
* [Generate the ICA](#ica) (~15 min) · <span style="color:#ffcc00">Checkpoint #2</span>
* [Remove the heartbeat and eye movements](#remove) (~10 min) · <span style="color:#ffcc00">Checkpoint #3</span>
* [Pre-process the empty room recording](#empty) (~5 min)

Then, the freesurfer steps take more time:
* [Cortical reconstruction with Freesurfer](#freesurfer) (~ 8 hr)
* [Co-registration](#coregistration) (~20 min)

Processing part (in development):
* [Normalize the conditions](#triggers) (~5 min)

Approx. times are indicative and can vary depending on the length of the processed data.

<a id="setup"></a>
## Setup

### Access Cajal

* From Citrix (remotely):

    Go to https://gateway.bcbl.eu/Citrix/BCBL_GatewayWeb/ and open **Terminator** (Linux Apps).
    
    Once on your virtual desktop, connect to cajal03: `vglconnect USERNAME@cajal03 -Y`, and enter your password.

* From a BCBL computer (locally):
    
    In the Start menu, open Cygwin-X > run XWin Server (if you cannot find it on your BCBL computer, open a ticket to install it).

    Two X icons will appear in the system tray. Right click on the icon with the green X > System tools > XTerm. These steps are necessary to obtain a GUI (running the following line on the regular Windows terminal will not work).
    
    Once on it, type in `ssh cajal03 -Y`, followed by your password.

### Setup the environment
Load **Python** using `module load python`.

Run **Visual Studio Code** using `code`.

### Pre-requisites
Be sure to run the code on an environment with the required Python modules installed: **numpy**, **matplotlib**, **mne** and **mne_bids**.

To install a Python module, use `py -m pip install MODULENAME` or `python -m pip install MODULENAME`. Note that to install **mne_bids**, the MODULENAME will be `mne-bids`.

The environment **megtrain (Python 3.11.6)** should contain the required modules.
(to modify it, `conda activate /bcbl/home/public/MEGtrain/Code/megtrain/megtrain/.conda`)

(modules needed in the venv: `numpy`, `matplotlib`, `mne`, `mne-bids`, `ipywidgets`, `ipyevents`, `nibabel`)

<a id="import"></a>
### Import sections
*Approx. time: 1 s*

In [ ]:
# Navigate in the OS and call folders
import os
import os.path as op

# Date and time
from datetime import datetime as dt

# Get the username automatically
import getpass

# Perform plots
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np

# MEG processing
import mne
from mne_bids import (
    write_raw_bids,
    read_raw_bids,
    BIDSPath,
    write_meg_calibration,
    write_meg_crosstalk,
    mark_channels
)

<a id="load"></a>
## Load the data

### Locate the subject data and define output path
*Approx. time: 1 s*

<span style="color:red">Contains a prompt asking the user to enter one of the subjects</span>, with the path automatically taken from the dictionary "subjects".

<span style="color:#99cc00">Creates: ``Derivatives/Preprocessing/sub-XX`` folder.</span>

In [ ]:
# Configuration
user = getpass.getuser()
path_data = op.join('/export/home', user, 'lab/MEG_EXPERIMENTS/EXPERIMENT_NAME/DATA/FIF/')

# Subjects: Write down the path to the subjects of your experiment here. 
# The path should contain the name of the first FIF file (without the .fif extension).
subjects = {"subject_01a": 'path_to_subject_01/session_01/block_01',
            "subject_01b": 'path_to_subject_01/session_02/block_01',
            "subject_02a": 'path_to_subject_02/session_01/block_01',
            "subject_02b": 'path_to_subject_02/session_02/block_01'}

subject = input("Subject name? ")
if subject in subjects.keys():
    subject_fif = subjects[subject] + ".fif"
    subject_number = subject[8:10]
    subject_session = "02" if subjects[subject].split("/")[0][-2:] == "_2" else "01"
    subject_eelbrain_code = "sub-" + subject_number
    visit = "6mt" if 'b' in subjects[subject].split("_")[1] else ""
else:
    raise Exception("Subject name not found.")

# Subject path
raw_fif_name = op.join(path_data, subject_fif)
print(f"Path of the selected subject: {str(raw_fif_name)}")

# Define the raw BIDS output path
path_output = op.join('/export/home', user, 'public/MEGtrain/')
path_bids = op.join(path_output, 'bids')

# Calibration and crosstalk files 
fine_cal_file = op.join(path_output,'scripts/maxfilter_parameters/sss_cal_3049.dat')
crosstalk_file = op.join(path_output,'scripts/maxfilter_parameters/ct_sparse.fif')

# Define the preprocessing output path
path_preprocessing = op.join(path_output, "derivatives/Preprocessing/")

path_derivatives = BIDSPath(
    subject = subject_number, 
    session = subject_session, 
    task = 'track', 
    run = "01",
    suffix = "meg",
    root = path_preprocessing
)
os.makedirs(path_derivatives.directory, exist_ok = True)

print(f"User: {user}")
print(f"Subject: {subject}")
print(f"Subject number: {subject_number}")
print(f"Subject session: {subject_session}")

### Create the output MNE path
*Approx. time: 1 m 30 s*

Reads the original FIF files and uniformizes the data by creating a BIDS structure in the path `path_output`. We also create the MEG fine calibration and cross-talk files, which are MEG files that are specific to the MEG acquistion machine, and describe its parameters. They allow to fine-tune the Maxwell filtering later on.

<span style="color:#99cc00">Creates: ``MEGtrain/sub-XX/ses-XX/`` folder, with:</span>

* ``sub-XX_ses-XX_scans.tsv``
* ``meg/sub-XX_ses-XX_coordsystem.json``
* ``meg/sub-XX_ses-XX_task-track_run-01_channels.tsv``
* ``meg/sub-XX_ses-XX_task-track_run-01_meg.json``
* ``meg/sub-XX_ses-XX_task-track_run-01_split-XX_meg.fif`` <span style="color:#99cc00">(multiple files)</span>
* ``meg/sub-XX_ses-XX_task-track_acq-calibration_meg.dat``
* ``meg/sub-XX_ses-XX_task-track_acq-crosstalk_meg.fif``

<span style="color:#99cc00">The files ``dataset_description.json``, ``participants.json``, and ``participants.tsv`` are also created or updated.
</span>

In [ ]:
# Read the FIF file
raw_fif = mne.io.read_raw_fif(raw_fif_name, preload = False)

# Define the BIDS path
path_bids_raw = BIDSPath(
    subject = subject_number, 
    session = subject_session, 
    task = 'track', 
    run = "01",
    datatype = "meg", 
    root = path_bids)

# Write the BIDS files
write_raw_bids(raw = raw_fif, bids_path = path_bids_raw, overwrite = True)

# Write calibration files in BIDS format
write_meg_calibration(fine_cal_file, path_bids_raw)
write_meg_crosstalk(crosstalk_file, path_bids_raw)

<a id="bad"></a>
## Bad channel detection
### Automatic detection
*Approx. time: 5 m*

In [ ]:
# Open the BIDS data
raw_bids = read_raw_bids(path_bids_raw)

# Define the "bads" channels to being empty
raw_bids.info['bads'] = []

# Run the automatic detection
auto_noisy_channels, auto_flat_channels, auto_scores = mne.preprocessing.find_bad_channels_maxwell(
    raw_bids,  # Raw data to process
    cross_talk = crosstalk_file, # MEG crosstalk file
    calibration = fine_cal_file,  # MEG fine-calibration file
    return_scores = True, 
    duration = 30,  # Duration of each window (in seconds)
    min_count = 10,  # Number of window appearances to be counted as bad channel
    verbose = True)

In [ ]:
print(f"Flat channels: {str(auto_flat_channels)}")
print(f"Bad channels: {str(auto_noisy_channels)}")

### Manual detection

Check the channels visually, to check if some channels weren't detected as bad automatically, or if there were false positives. 

Tips: "Zooming out" allows to detect noisy (bad) channels, while "Zooming in" allows to detect flat channels.

Click on the name of a channel in the left part of the screen to mark them as bad.

In [ ]:
raw_bids.plot(lowpass = 30, highpass = 1, overview_mode = "hidden")

You can also write the bad channels in the dictionary below to preserve a trace of them and add them to the bad channels.

In [ ]:
# Manually writing the bad channels
bad_channels_manual = {"subject_01a": ["MEG2413", "MEG2623"],
                       "subject_01b": ["MEG1912"],
                       "subject_02a": ["MEG2623"],
                       "subject_02b": ["MEG0312", "MEG0711"]}

raw_bids.info["bads"] = bad_channels_manual[subject]

# Combining the automatically found bad and flat channels to the manual ones
raw_bids.info["bads"] += auto_noisy_channels + auto_flat_channels

print(f"Bad channels: {str(raw_bids.info['bads'])}")

# Once the bad channels are defined, we save them in the file itself
mark_channels(path_bids_raw, ch_names = raw_bids.info['bads'], status = "bad")

<a id="maxwell"></a>
## Maxwell filtering and Signal-Space Separation
*Approx. time: 15 m*

### Compute the head positions

First, we compute the continuous head positions thanks to the coils. These allow us to compensate for the head movements and helps to clean the signal.

In [ ]:
# Extract continuous head position indicators (cHPI)
chpi_amplitudes = mne.chpi.compute_chpi_amplitudes(raw_bids, t_window = 0.5, t_step_min = 0.1)

# Estimate the coils locations
chpi_locs = mne.chpi.compute_chpi_locs(raw_bids.info, chpi_amplitudes)

# Estimate the head positions
head_pos = mne.chpi.compute_head_pos(raw_bids.info, chpi_locs, verbose = True)

# Save the head positions
path_pos = op.join(path_derivatives.directory, path_derivatives.basename + ".pos")
mne.chpi.write_head_pos(path_pos, head_pos)

### Run the Maxwell filter
Now, we run a Maxwell filter and a Signal-Space Separation (SSS) to get a signal with reduced environmental noise and head movement compensation. These two techniques are performed together by calling the filter function from MNE. 

It is important to have detected the bad channels beforehands, as bad sensors impact drastically the filtering of their neighbors.

In [ ]:
# Run the Maxfilter
raw_sss = mne.preprocessing.maxwell_filter(
    raw_bids, 
    cross_talk = crosstalk_file,  # MEG crosstalk file
    calibration = fine_cal_file,  # MEG fine-calibration file
    head_pos = head_pos,  # Head positions
    st_duration = 10, # Duration of the buffer (in seconds) 
    st_correlation = 0.98  # Correlation limit between inner and outer subspaces used to reject overlapping intersecting inner/outer signals during spatiotemporal SSS.
)

### Downsampling the data
Next, we downsample the data at 200 Hz.

The original data from the MEG is sampled at 1000 Hz, with 306 channels. It is possible to encounter an integer overflow in the MNE Python package if the amount of indices in the data matrix reach 2<sup>31</sup> (2 147 483 648). This will be reached after recording more than 1 hour, 56 minutes and 57 seconds of raw recording (7017.92 seconds). To avoid this, we downsample the data at 200 Hz, which also allows to speed up the next steps.

In [ ]:
# Deleting raw_bids to save memory
del raw_bids

# Downsampling the data at 200 Hz
raw_downsampled_sss = raw_sss.resample(sfreq = 200) # Automatic low-pass at 100 Hz

### Saving the data

*Approx. time: 1 m*

<span style="color:#99cc00">Creates ``derivatives/Preprocessing/sub-XX/ses-XX/sub-XX_ses-XX_task-track_run-01_sss.fif`` (multiple files).</span>

In [ ]:
# We save the filtered data
path_downsampled_sss = op.join(path_derivatives.directory, path_derivatives.basename.replace("meg", "sss") + ".fif")
raw_downsampled_sss.save(path_downsampled_sss, overwrite = True)

***
<span style="color:#ffcc00">**Checkpoint #1**</span>

<span style="color:#ffcc00">You can safely close the pipeline here and come back later. To start back, you will only need to run the [import](#import) and [path defining](#load) steps.</span>

<a id="ica"></a>
## Generate the ICA

### Restore the data from a file
Only perform this line if you don't want to re-do the previous steps.

In [ ]:
path_downsampled_sss = op.join(path_derivatives.directory, path_derivatives.basename.replace("meg", "sss") + ".fif")
raw_downsampled_sss = mne.io.read_raw_fif(path_downsampled_sss, preload = True)

### Run the ICA

*Approx. time: 15 m*

The ICA tries to detect automatically components in the signal emerging from the data of all the sensors. We use it to detect the heartbeat and the horizontal and vertical eye movements components that we will then reject from the data.

In [ ]:
# Apply a band-pass filter
raw_for_ica = raw_downsampled_sss.copy().filter(l_freq = 0.5, h_freq = 30)

# Create the ICA object
ica = mne.preprocessing.ICA(  # Create an ICA object
    n_components = 20,  # Define how many components we want in our ICA
    random_state = 97,   # Define a seed for the randomness generator to get consistent results
    method = "picard",  # Makes the ICA faster
    fit_params = dict(ortho = True, extended = True)
)

# Fit the ICA to our data
ica.fit(raw_for_ica, picks = 'meg')

### Save the ICA

*Approx. time: 1 s*

<span style="color:#99cc00">Creates ``derivatives/Preprocessing/sub-XX/ses-XX/sub-XX_ses-XX_task-track_run-01_ica_solution.fif`` (multiple files).</span>

In [ ]:
path_ica_solution = op.join(path_derivatives.directory, path_derivatives.basename.replace("meg", "ica_solution") + ".fif")
ica.save(path_ica_solution, overwrite = True)

***
<span style="color:#ffcc00">**Checkpoint #2**</span>

<span style="color:#ffcc00">You can safely close the pipeline here and come back later. To start back, you will only need to run the [import](#import) and [path defining](#load) steps.</span>

<a id="remove"></a>
## Remove the heartbeat and eye movements

### Restore the ICA and filtered data from file
*Approx. time: 2 m*

Only perform this line if you don't want to re-do the previous steps.

In [ ]:
# Load raw for ICA data
path_downsampled_sss = op.join(path_derivatives.directory, path_derivatives.basename.replace("meg", "sss") + ".fif")
raw_downsampled_sss = mne.io.read_raw_fif(path_downsampled_sss, preload = True)
raw_for_ica = raw_downsampled_sss.copy().filter(l_freq = 0.5, h_freq = 30)

# Load the ICA
path_ica_solution = op.join(path_derivatives.directory, path_derivatives.basename.replace("meg", "ica_solution") + ".fif")
ica = mne.preprocessing.read_ica(path_ica_solution)

### Plot the ICA
The ICA should detect the heartbeat and the eye movements (both vertical and horizontal) among the first components, as they are strong signals that are collected by most, if not all sensors.
We can take a look at the output from the ICA two ways: by plotting the time course of the components, or by plotting the scalp field distribution of the components.

Here is [a resource](https://labeling.ucsd.edu/tutorial/labels) to learn to identify the components properly.

#### Plot the scalp field distrubution
The scalp map of the **heartbeat** will look like that of a very far dipole and so will look almost like a linear gradient. 

The **vertical eye movements** have a scalp topography that is oriented up and down.

The **horizontal eye movements** have a scalp topography that can be modeled as a dipole with left/right with positive values on one side and negative on the other.

In [ ]:
plot = ica.plot_components()

#### Plot the time course
The **heartbeat** component should look like a regular, strong pulse.

The **vertical eye movements** component should show clear spikes relatively frequently due to eye blinks, with stable movements in between, of generally lower amplitude than the other components.

The **horizontal eye movements** component should have intervals of relative stability with occasional and very fast transitions to different values due to visual scanning (if the task involves it).

Both the vertical and horizontal eye movements should be below 5 Hz.

In [ ]:
mne.viz.set_browser_backend("qt")
raw_for_ica.load_data()
ica.plot_sources(raw_for_ica, show_scrollbars = False)

### Check EOG and ECG channels

We now need to ensure that the channels for the ECG, VEOG and HEOG are the good ones for our participant. In some participants, the channels can be different from the default ones (HEOG: EOG061, VEOG: EOG062, ECG: ECG063).

Here, we plot the channels. If the channels do not look like the good ones, we can use `raw.plot()` to try to find the proper channels.

In [ ]:
mne.viz.set_browser_backend("qt")

# Define the EOG and ECG channels
# Check that they match the actual EOG/ECG info!
eog_ecg_channels = {"default": {"HEOG": "EOG061", "VEOG": "EOG062", "ECG": "ECG063"},
                    "subject_01a": {"HEOG": "EOG061", "VEOG": "EOG062", "ECG": "EEG005"},
                    "subject_01b": {"HEOG": "EOG061", "VEOG": "EOG062", "ECG": "ECG063"},
                    "subject_02a": {"HEOG": "EOG061", "VEOG": "EOG062", "ECG": "ECG063"},
                    "subject_02b": {"HEOG": "EOG061", "VEOG": "EOG062", "ECG": "ECG063"}}

heog = eog_ecg_channels[subject]["HEOG"]  # Default: E0G061
veog = eog_ecg_channels[subject]["VEOG"]  # Default: E0G062
ecg = eog_ecg_channels[subject]["ECG"]  # Default: ECG063

# Check using this line
channels_of_interest = raw_for_ica.copy().pick_channels([heog, veog, ecg])
channels_of_interest.plot()

### Plot the raw data
Only use if you think that the channels selected don't look like the good ones.

In [ ]:
raw_for_ica.plot(overview_mode = "hidden")

### Find the ICA components matching the EOG and ECG
*Approx. time: 5 m*

In [ ]:
# Analyze EOG/ECG channels
heog_indices, heog_scores = ica.find_bads_eog(raw_for_ica, ch_name = heog)
veog_indices, veog_scores = ica.find_bads_eog(raw_for_ica, ch_name = veog)
ecg_indices, ecg_scores = ica.find_bads_ecg(raw_for_ica, ch_name = ecg)

# Reject ICAs containing the EOG and ECG information from all channels
ica.exclude = list(set(veog_indices + heog_indices + ecg_indices))
print(f"Rejected ICAS: {str(ica.exclude)}")

### Select the components matching the EOG and ECG

Note: all of the next sub-steps will allow you to take an informed decision whether to keep or change the components to remove. It is not necessary to run them every time, but all inform you as to what type of information you would be removing by selecting some components.

#### Plot the correlation scores for each channel/ICA
*Approx. time: 1 s*

In [ ]:
plt.rcParams["figure.figsize"] = (15, 3)
fig, axes = plt.subplots(1, 3)
plt.setp(axes, xticks=[i for i in range(ica.n_components)], ylim=[0, 1])

plt.sca(axes[0])
plt.bar([i for i in range(ica.n_components)], np.abs(heog_scores), color=["blue" if i in heog_indices else "black" for i in range(ica.n_components)])
plt.title("HEOG")

plt.sca(axes[1])
plt.bar([i for i in range(ica.n_components)], np.abs(veog_scores), color=["green" if i in veog_indices else "black" for i in range(ica.n_components)])
plt.title("VEOG")

plt.sca(axes[2])
plt.bar([i for i in range(ica.n_components)], np.abs(ecg_scores), color=["red" if i in ecg_indices else "black" for i in range(ica.n_components)])
plt.title("ECG")

fig.tight_layout()
plt.show()

#### Plot the explained variance of each component
This allow to know how much variance in the data is explained by each component, in order to take a decision as to whether remove a component or not. Removing a component with a lot of variance will have more impact on the overall data.

In [ ]:
# Unitize variances explained by PCA components, so the values sum to 1
pca_explained_variances = ica.pca_explained_variance_ / ica.pca_explained_variance_.sum()

# Now extract the variances for those components that were used to perform ICA
ica_explained_variances = pca_explained_variances[:ica.n_components_]

axes = plt.gca()
axes.set_xticks([i for i in range(ica.n_components)])
axes.set_ylim([0, 1])
axes.yaxis.set_major_formatter(mtick.PercentFormatter())

plt.bar([i for i in range(ica.n_components)], ica_explained_variances)

for i in range(ica.n_components):
        plt.text(i,  # x position
                 ica_explained_variances[i] + .05,  # y position
                 str(round(ica_explained_variances[i]*100, 2)) + " %",  # Text to plot
                 ha = 'center')
plt.show()

#### Compare the rejected components to the EOG and ECG channels

In [ ]:
mne.viz.set_browser_backend("qt")
plot = ica.plot_sources(raw_for_ica, picks=ica.exclude, start=0., stop=30., show_scrollbars=False)

#### Manually reject components
If the previous step has not rejected an ICA, or has partially rejected the ICAs, you can manually reject components here.

In [ ]:
# Add components
ica.exclude += []

# Remove components
ica.exclude = [comp for comp in ica.exclude if comp not in []]

#### Superimpose the data before and after removing a specific component
*Approx. time: 10 s*

Shows what some selected channels will look like after removing a selected component.

In [ ]:
plt.rcParams["figure.figsize"] = (15, 5)
plot = ica.plot_overlay(
    raw_for_ica,  # The original data
    exclude = ica.exclude,  # The ICA component to remove
    picks = 'meg',   # The channels to display
    start = 0.,  # First timestamp of the display window (seconds)
    stop = 30.,  # Last timestamp of the display window (seconds)
)

### Reject the components
*Approx. time: 20 s*

In [ ]:
raw_filtered_ica = ica.apply(raw_for_ica)

### Save the ICA-filtered data
*Approx. time: 1 m*

<span style="color:#99cc00">Creates ``derivatives/Preprocessing/sub-XX/ses-XX/sub-XX_ses-XX_tast-track_run-01_ica.fif`` (multiple files).</span>

In [ ]:
path_ica = op.join(path_derivatives.directory, path_derivatives.basename.replace("meg", "ica") + ".fif")
raw_filtered_ica.save(path_ica, overwrite = True)

### Save the pictures showing the ICA components
*Approx. time: 5 s*

<span style="color:#99cc00">Creates the following image files in the folder ``derivatives/Preprocessing/sub-XX/ses-XX/``:

* ``sub-XX_ses-XX_task-track_run-01_ecg_scores.png``
* ``sub-XX_ses-XX_task-track_run-01_heog_scores.png``
* ``sub-XX_ses-XX_task-track_run-01_veog_scores.png``
* ``sub-XX_ses-XX_task-track_run-01_ica_all_components.png``
* ``sub-XX_ses-XX_task-track_run-01_ica_rejected_components.png``

</span>

In [ ]:
# All components
f1 = ica.plot_components()
path_ica_all_components = op.join(path_derivatives.directory, path_derivatives.basename.replace("meg", "ica_all_components") + ".png")
f1.savefig(path_ica_all_components, dpi=300)

# Rejected components
f2 = ica.plot_components(picks=ica.exclude)
path_ica_rejected_components = op.join(path_derivatives.directory, path_derivatives.basename.replace("meg", "ica_rejected_components") + ".png")
f2.savefig(path_ica_rejected_components, dpi=300)

# HEOG channel
f3 = ica.plot_scores(heog_scores, show=False)
path_ica_heog = op.join(path_derivatives.directory, path_derivatives.basename.replace("meg", "heog_scores") + ".png")
f3.savefig(path_ica_heog, dpi=300)

# VEOG channel
f4 = ica.plot_scores(veog_scores, show=False)
path_ica_veog = op.join(path_derivatives.directory, path_derivatives.basename.replace("meg", "veog_scores") + ".png")
f4.savefig(path_ica_veog, dpi=300)

# ECG channel
f5 = ica.plot_scores(ecg_scores, show=False)
path_ica_ecg = op.join(path_derivatives.directory, path_derivatives.basename.replace("meg", "ecg_scores") + ".png")
f5.savefig(path_ica_ecg, dpi=300)

#### Save ICA in eelbrain format
If you want to perform a mTRF analysis, you may want to save your data for eelbrain.

<span style="color:#99cc00">Creates ``eelbrain/meg/sub-XX/sub-XX_track-raw.fif`` (multiple files).</span>

In [ ]:
path_eelbrain_meg = op.join(path_output, 'eelbrain', 'meg')
path_eelbrain = op.join(path_eelbrain_meg, subject_eelbrain_code, subject_eelbrain_code + "_track-raw.fif")
raw_eelbrain = raw_filtered_ica.copy().pick(['meg','stim', 'misc', 'chpi'])  # We remove the EEG channels (in the case they exist) as they conflict with eelbrain
raw_eelbrain.save(path_eelbrain, overwrite = True, verbose = False)

#### Take a look at the clear data
We can plot the raw data before and after ICA to see the difference.

In [ ]:
mne.viz.set_browser_backend("qt")  # Put "matplotlib" if you prefer to get it inline or "qt" if you want it in a window
plot = raw_downsampled_sss.plot(show_scrollbars = False, overview_mode = "hidden")
plot = raw_filtered_ica.plot(show_scrollbars = False, overview_mode = "hidden")

***
<span style="color:#ffcc00">**Checkpoint #3**</span>

<span style="color:#ffcc00">You can safely close the pipeline here and come back later. To start back, you will only need to run the [import](#import) and [path defining](#load) steps.</span>

<a id="empty"></a>
## Pre-process the empty-room recording

### Find the best empty room measurement

*Approx. time: 1 s*

We first need to find the temporally closest empty room from the folder `lab/MEG_EXPERIMENTS/EMPTYROOM/DATA/`.

The code below prints the recording date of the current subject.

In [ ]:
info = mne.io.read_info(raw_fif_name, verbose = False)
date_measurement = info["meas_date"].replace(tzinfo = None)
print(f"Measurement date: {date_measurement.strftime('%A %d %B %Y %H:%M:%S')}")

The code below automatically finds the closest empty room recording.

In [ ]:
# Get all of the recordings folders
path_empty_room = op.join('/export/home', user, 'lab/MEG_EXPERIMENTS/EMPTYROOM/DATA/')
subfolders_empty_room = os.listdir(path_empty_room)
subfolders_empty_room.remove("other")

empty_room_recordings = []
for subfolder in subfolders_empty_room:
    subfolder_recordings = os.listdir(op.join(path_empty_room, subfolder))
    for recording in subfolder_recordings:
        if op.isdir(op.join(path_empty_room, subfolder, recording)):
            empty_room_recordings.append(subfolder + "/" + recording)

# Find the smallest difference of days
smallest_diff = None
selected_recording = None
for folder in empty_room_recordings:
    recording = folder.split("/")[1]
    if recording != ".directory":
        date_empty_room = dt(int(recording[0:2]) + 2000, int(recording[2:4]), int(recording[4:6]))  # Turn the folder name into a date
        day_diff = date_measurement - date_empty_room
        if selected_recording is None or abs(smallest_diff) > abs(day_diff):  # If we find a smaller difference of days than the previous, we set it
            smallest_diff = day_diff
            selected_recording = folder

# Print the selected empty room
folder_name = selected_recording.split("/")[1]
empty_room_date = dt(int(folder_name[0:2]) + 2000, int(folder_name[2:4]), int(folder_name[4:6])).strftime("%A %d %B %Y")
if smallest_diff.days > 0:
    print(f"Selected empty room recording: {selected_recording}, recorded {empty_room_date} ({smallest_diff.days} day(s) before the subject recording).")
elif smallest_diff.days < 0:
    print(f"Selected empty room recording: {selected_recording}, recorded {empty_room_date} ({abs(smallest_diff.days)} day(s) after the subject recording).")
else:
    print(f"Selected empty room recording: {selected_recording}, recorded {empty_room_date} (the same day of the subject recording).")

path_selected_er = op.join(path_empty_room, selected_recording)
path_empty_room_fif = op.join(path_selected_er, os.listdir(path_selected_er)[0])
print(f"Path: {path_empty_room_fif}")

### Save the empty room recording in BIDS format

*Approx. time: 15 s*

<span style="color:#99cc00">Creates the following files in the ``MEGtrain/sub-XX/ses-XX/`` folder:</span>

* ``meg/sub-XX_ses-XX_task-emptyroom_run-01_channels.tsv``
* ``meg/sub-XX_ses-XX_task-emptyroom_run-01_meg.json``
* ``meg/sub-XX_ses-XX_task-emptyroom_run-01_split-XX_meg.fif``</span>

<span style="color:#99cc00">The files ``sub-XX_ses-XX_scans.tsv`` and ``meg/sub-XX_ses-XX_coordsystem.json`` are also updated.</span>

In [ ]:
# Read the FIF file
empty_room_fif = mne.io.read_raw_fif(path_empty_room_fif, preload = False)

# We remove any digitization point that could be saved from a previous session, so we use a proper empty room
empty_room_fif.set_montage(None)

# Write the BIDS files
path_bids_empty_room = BIDSPath(
    subject = subject_number, 
    session = subject_session, 
    task = 'emptyroom', 
    run = "01",
    datatype = "meg", 
    root = path_bids)

write_raw_bids(raw = empty_room_fif, bids_path = path_bids_empty_room, overwrite = True)

# We delete the empty_room_fif to free some memory
del empty_room_fif

### Bad channel detection

#### Automatic detection
*Approx. time: 5 s*

In [ ]:
# Open the BIDS data
raw_bids_empty_room = read_raw_bids(path_bids_empty_room)

# Define the "bads" channels to being empty
raw_bids_empty_room.info['bads'] = []

# Run the automatic detection
auto_noisy_channels_er, auto_flat_channels_er, auto_scores_er = mne.preprocessing.find_bad_channels_maxwell(
    raw_bids_empty_room,  # Raw data to process
    cross_talk = crosstalk_file, 
    calibration = fine_cal_file,
    return_scores = True, 
    coord_frame = "meg",  # Necessary for empty room recordings
    duration = 30,  # Duration of each window (in seconds)
    min_count = 10,  # Number of window appearances to be counted as bad channel
    verbose = True,
    )

print(f"Flat channels: {str(auto_flat_channels_er)}")
print(f"Bad channels: {str(auto_noisy_channels_er)}")

#### Manual detection

In [ ]:
raw_bids_empty_room.plot(lowpass = 30, highpass = 1, overview_mode = "hidden")

#### Combination

In [ ]:
# Manually writing the bad channels
bad_channels_er_manual = {"subject_06a": ["MEG1122", "MEG2623"],
                          "subject_06b": [],
                          "subject_07a": ["MEG1122", "MEG2623"],
                          "subject_07b": ["MEG0312"]}

raw_bids_empty_room.info["bads"] = bad_channels_er_manual[subject]

# Combining the automatically found bad and flat channels to the manual ones
raw_bids_empty_room.info["bads"] += auto_noisy_channels_er + auto_flat_channels_er

print("Bad channels: " + str(raw_bids_empty_room.info["bads"]))

# Once the bad channels are defined, we save them in the file itself
mark_channels(path_bids_empty_room, ch_names = raw_bids_empty_room.info['bads'], status = "bad")

### Maxwell filtering and Signal-Space Separation
*Approx. time: 1 m*

<span style="color:#99cc00">Creates ``derivatives/Preprocessing/sub-XX/ses-XX/sub-XX_ses-XX_task-emptyroom_run-01_sss.fif``.</span>

In [ ]:
# Define the saving path
path_derivatives_er = BIDSPath(
    subject = subject_number, 
    session = subject_session, 
    task = 'emptyroom', 
    run = "01",
    suffix = "meg",
    root = path_preprocessing
)

# Run the Maxfilter
raw_er_sss = mne.preprocessing.maxwell_filter(
    raw_bids_empty_room, 
    cross_talk = crosstalk_file, 
    calibration = fine_cal_file,
    coord_frame = "meg",  # Necessary for empty room recordings
    st_duration = 10,  # Duration of the buffer (in seconds) 
    st_correlation = 0.98  # Correlation limit between inner and outer subspaces used to reject overlapping intersecting inner/outer signals during spatiotemporal SSS.
)

# Downsampling the data
raw_er_downsampled_sss = raw_er_sss.resample(sfreq = 200)  # Automatic low-pass at 100 Hz

# Saving the downsampled data
path_downsampled_sss_er = op.join(path_derivatives_er.directory, path_derivatives_er.basename.replace("meg", "sss") + ".fif")
raw_er_downsampled_sss.save(path_downsampled_sss_er, overwrite = True, verbose = False)

### Save ICA in eelbrain format

<span style="color:#99cc00">Creates ``eelbrain/meg/sub-XX/sub-XX_emptyroom-raw.fif`` (multiple files).</span>

In [ ]:
path_eelbrain_meg = op.join(path_output, 'eelbrain', 'meg')
path_eelbrain_er = op.join(path_eelbrain_meg, subject_eelbrain_code, subject_eelbrain_code + "_emptyroom-raw.fif")
raw_er_eelbrain = raw_er_downsampled_sss.copy().pick(['meg','stim'])  # We remove the EEG channels (in the case they exist) as they conflict with eelbrain
raw_er_eelbrain.save(path_eelbrain_er, overwrite = True, verbose = False)

<a id="freesurfer"></a>
## Cortical reconstruction with Freesurfer
*Approx. time: 8 h*

The next step is to get a reconstruction of the cortex using Freesurfer. This step is by far the longest. **This step must be ran on cajal01**.

From Citrix, open **Terminator** (Linux Apps). Once on your virtual desktop, connect to cajal03: `vglconnect USERNAME@cajal01 -Y`, and enter your password.

Note: to paste code in bash, you have by default to press Ctrl+Shift+V. You can change this in the parameters of the console, in the tab named "Keybindings" by something less stupid, like Ctrl+V.

### Load Freesurfer
`module load freesurfer/6.0.0`

### Create the necessary Freesurfer variables
Output directory:

`export SUBJECTS_DIR=~/public/MEGtrain/eelbrain/mri`

Subject name (will be the name of the folder created in the output directory)

`export SUBJECT=sub-01`

### Get the MRI data path and run the code

<span style="color:#99cc00">This step creates ``eelbrain/mri/sub-XX/`` and all its contents (subfolders ``label``, ``mri``, ``scripts``, ``stats``, ``surf``, ``tmp``, ``touch`` and ``trash``). ``tmp`` and ``trash`` folders are empty.</span>

**Important note:** Bash is a very old and stupidly designed language that is scared by blank spaces. One of the folders in the full path to the T1 is named `CRANEO_FUNCIONAL - 1`, with two spaces around the hyphen. While some command lines allow to bypass this by putting backslashes before each space (`CRANEO_FUNCIONAL\ -\ 1`) or putting simple or double quotes around the path (`"CRANEO_FUNCIONAL - 1"`), the Freesurfer command line inexplicably doesn't manage to understand that the path contains spaces. As a workaround, **delete the spaces from the path** - you cannot do that from the lab folder, but the T1s have been copied to `~/public/MEGtrain/eelbrain/mri/t1`. There, you can rename the folder `CRANEO_FUNCIONAL - 1` by `CRANEO_FUNCIONAL-1`.

Once you did that, you have many possibilities:

* Get into the path containing the data

    `cd "~/public/MEGtrain/eelbrain/mri/t1/EXPERIMENT_NAME/CRANEO_FUNCIONAL-1/t1_mprage_sag_p2_1iso_MGH_6"`

    In that case, run the code that way:

    `nice recon-all -s $SUBJECT -i IM-0004-0001.dcm -autorecon-all -mail YOURMAILADDRESS`

* Put the whole path in the code line (this line will run independently of your current working directory):

    `nice recon-all -s $SUBJECT -i ~/public/MEGtrain/eelbrain/mri/t1/EXPERIMENT_NAME/CRANEO_FUNCIONAL-1/t1_mprage_sag_p2_1iso_MGH_6/IM-0004-0001.dcm -autorecon-all -mail YOURMAILADDRESS`

Note: You can run multiple jobs in parallel by pressing Ctrl+Shift+T on Terminator: this will open a new tab where you can run another participant. However, keep in mind the resources of the cluster are shared between the BCBL members. You can check how busy the cluster currently is by typing `top`.

<a id="coregistration"></a>
## Co-registration (from bash)
*Approx. time: 20 m*

The final step is to perform a co-registration between the MRI and the MEG space. We will once again need to use Freesurfer, on cajal01.

### Load the environment
    module load freesurfer/6.0.0
    module load python/python3.6
    source activate eelbrain
    vglrun ipython

### Prepare your variables
You are now in iPython. Start by importing the modules you will need:

    import mne
    import os

Then, define your variables:

    user = USERNAME
    path_output = os.path.join('/export/home', user, 'public/MEGtrain/')
    subjects_dir = os.path.join(path_output, 'eelbrain/mri/')
    os.environ["SUBJECTS_DIR"] = subjects_dir
    subject = 'sub-01'

### Create the watershed folder
*Approx. time: 8 m*

<span style="color:#99cc00">This step creates ``eelbrain/mri/sub-XX/bem`` and all its contents:

* ``brain.surf``
* ``inner_skull.surf``
* ``outer_skin.surf``
* ``outer_skull.surf``
* ``sub-XX-head.fif``
* ``watershed/sub-XX_brain_surface``
* ``watershed/sub-XX_inner_skull_surface``
* ``watershed/sub-XX_outer_skin_surface``
* ``watershed/sub-XX_outer_skull_surface``
* ``watershed/ws.mgz``

</span>

``mne.bem.make_watershed_bem(subject = subject, overwrite = True)``

### Create the head-dense.fif file
*Approx. time: 1 m*

<span style="color:#99cc00">This step creates the following files in the ``eelbrain/mri/sub-XX/bem`` folder:

* ``sub-XX-head-dense.fif``
* ``sub-XX-head-medium.fif``
* ``sub-XX-head-sparse.fif``

</span>

``mne.bem.make_scalp_surfaces(subject = subject, overwrite = True, force = True)``

### Visualize the hemispheres
You can now visualize each hemisphere within the head.

    Brain = mne.viz.get_brain_class()  # Create a Brain object
    brain = Brain(subject, 
                  hemi = 'both',  # The hemisphere to visualize (`lh`, `rh`, `both`, or `split`)
                  subjects_dir = subjects_dir, 
                  size = (800, 600)  # The size of the window in pixels
    )
    brain.add_annotation('aparc.a2009s', borders = False)
    brain.add_head(dense = True)

### Perform the co-registration
*Approx. time: 5 m per session*

<span style="color:#99cc00">This step creates the following files:

* ``eelbrain/mri/sub-XX/bem/sub-XX-fiducials.fif``
* ``bids/sub-XX/ses-XX/sub-XX_ses-XX_task-track_run-01_trans.fif``

</span>

Finally, you can perform the co-registration with the head image.

To create the transform from head to MRI coordinate frame:

    mne.gui.coregistration(subject = subject, subjects_dir = subjects_dir)

This opens a GUI allowing you to align the MEG Polhemus coordinates to the MRI.

* Define the fiducials and lock them (the nasion should be centered between the eyes, while the RPA and LPA should be at the entrance of the ear, just touching the tragus).
* Save the fiducials.

**For each session:**
* Under Info source with digitization, select the path to the first split fif file of the session, e.g. `/export/home/USERNAME/public/MEGtrain/bids/sub-06/ses-01/meg/sub-06_ses-01_task-track_run-01_split-01_meg.fif` (it shouldn't matter to run the co-registration on the raw file or the pre-processed file as the coordinates are left untouched).
* Align the MEG dots with the MRI head using the translation and rotation parameters (on the left - generally a x-rotation of at least 30° is necessary).
* Save the HEAD <> MRI transform under `/MEGtrain/bids/sub-XX/ses-XX/sub-XX_ses-XX_task-track_run-01_trans.fif`

<a id="coregistrationvsc"></a>
## Co-registration (run from Visual Studio Code, works partially)
*Approx. time: 20 m*

The final step is to perform a co-registration between the MRI and the MEG space. 

Run this on **cajal01**. 

If you run this from Visual Studio Code, you may need to install the pre-version for Pylance and Python (by opening Extensions in the lateral menu). Beware that doing so will mean that you will have to reinstall Python on cajal03.

### Setup the environment
Load **Python** using `module load python`.

Load **Freesurfer** using `module load freesurfer/6.0.0`.

Run **Visual Studio Code** using `code`.

### Prepare your variables (works in VSC)
If you closed this file since the pre-processing, re-run the two first code blocks ([Import sections](#setup) and [Locate the subject data and define output path](#load) - enter the subject you want to process there). Then, prepare the variables needed for the co-registration:

In [ ]:
fs_subject = "sub-" + str(subject_number)
subjects_dir = op.join(path_output, 'eelbrain/mri/')
os.environ["SUBJECTS_DIR"] = subjects_dir
os.environ["MESA_GL_VERSION_OVERRIDE"] = "3.3"
os.environ["MNE_3D_OPTION_MULTI_SAMPLES"] = "1"

### Create the watershed folder (works in VSC)
*Approx. time: 8 m*

<span style="color:#99cc00">Creates: ``eelbrain/mri/sub-XX/bem`` folder, with:
* ``brain.surf``
* ``inner_skull.surf``
* ``outer_skin.surf``
* ``outer_skull.surf``
* ``sub-XX-head.fif``
* ``watershed/sub-XX_brain_surface``
* ``watershed/sub-XX_inner_skull_surface``
* ``watershed/sub-XX_outer_skin_surface``
* ``watershed/sub-XX_outer_skull_surface``
* ``watershed/ws.mgz``
</span>

In [ ]:
mne.bem.make_watershed_bem(subject = fs_subject, overwrite = True)

### Create the head-dense.fif file (works in VSC)
*Approx. time: 1 min*

<span style="color:#99cc00">This step creates the following files in the ``eelbrain/mri/sub-XX/bem`` folder:

* ``sub-XX-head-dense.fif``
* ``sub-XX-head-medium.fif``
* ``sub-XX-head-sparse.fif``

</span>

In [ ]:
mne.bem.make_scalp_surfaces(subject = fs_subject, overwrite = True, force = True)

### Visualize the hemispheres (does NOT work in VSC)
You can now visualize each hemisphere within the head.

In [ ]:
Brain = mne.viz.get_brain_class()  # Create a Brain object
brain = Brain(fs_subject, 
        hemi = 'both',  # The hemisphere to visualize (`lh`, `rh`, `both`, or `split`)
        subjects_dir = subjects_dir, 
        size = (800, 600)  # The size of the window in pixels
)
brain.add_annotation('aparc.a2009s', borders = False)
brain.add_head(dense = True)

### Perform the co-registration (does NOT work in VSC)
*Approx. time: 5 m per session*

<span style="color:#99cc00">This step creates the following files:

* ``eelbrain/mri/sub-XX/bem/sub-XX-fiducials.fif``
* ``bids/sub-XX/ses-XX/sub-XX_ses-XX_task-track_run-01_trans.fif``

</span>

Finally, you can perform the co-registration with the head image.

To create the transform from head to MRI coordinate frame:

In [ ]:
mne.gui.coregistration(subject = fs_subject, subjects_dir = subjects_dir)

This opens a GUI allowing you to align the MEG Polhemus coordinates to the MRI.

* Define the fiducials and lock them (the nasion should be centered between the eyes, while the RPA and LPA should be at the entrance of the ear, just touching the tragus).
* Save the fiducials.

**For each session:**
* Under Info source with digitization, select the path to the first split fif file of the session, e.g. `/export/home/USERNAME/public/MEGtrain/bids/sub-06/ses-01/meg/sub-06_ses-01_task-track_run-01_split-01_meg.fif` (it shouldn't matter to run the co-registration on the raw file or the pre-processed file as the coordinates are left untouched).
* Align the MEG dots with the MRI head using the translation and rotation parameters (on the left - generally a x-rotation of at least 30° is necessary).
* Save the HEAD <> MRI transform under `/MEGtrain/bids/sub-XX/ses-XX/sub-XX_ses-XX_task-track_run-01_trans.fif`